In [6]:
#coding:utf-8
import glob
import os
import os.path
import sys
import subprocess
import re
import shutil
import pickle
import numpy as np
import tensorflow as tf
from tqdm import tqdm


class PrepareDataset(object):
    
    PATH_AVDATA = '/media/ikesan009/B418B4D718B499B6/CENSREC/avdata'
    PATH_SAVE = '/media/ikesan009/B418B4D718B499B6/research/CENSREC/dataset'
    PATH_LABEL_TABLE = '/media/ikesan009/B418B4D718B499B6/research/CENSREC/lib/label_table'
    PATH_CONFIG = '/media/ikesan009/B418B4D718B499B6/research/CENSREC/lib/config.hcopy'
    MFCC_DIM = 42
    LEN_SEQ = 300
    
    #dirで指定されたパスが存在しない場合ディレクトリ作成
    def make_dir(self,dir,format=False):
        if not os.path.exists(dir):
            os.makedirs(dir)
        if format and os.path.exists(dir):
            shutil.rmtree(dir)
            
    #コマンドライン操作
    def run_command(self, cmd):
        out = subprocess.run(cmd, stdout=subprocess.PIPE)
        #print(out.stdout.decode())

    #HCopy操作
    def run_hcopy(self, path_config, path_wav, path_mfc):
        cmd = ['HCopy', '-C', path_config, path_wav, path_mfc]
        self.run_command(cmd)
        
    #HList操作
    def run_hlist(self, n_dim, path_mfc, path_txt):
        cmd = ['HList', '-i', str(n_dim), path_mfc]
        out = subprocess.run(cmd, stdout=subprocess.PIPE)
        with open(path_txt, mode='w', encoding='utf-8') as f:
            f.write(out.stdout.decode())
        
    #ゼロパディング
    def zero_padding(self, x, len_seq, dim_step):
        size_target = len_seq * dim_step
        x = np.append(x, np.zeros(size_target - x.size))
        return x
        
    #ファイル移動など
    def prepare_dataset(self):
        print('running prepare_dataset...')
        pbar1 = tqdm(os.listdir(self.PATH_AVDATA))
        for dir1 in pbar1:
            pbar1.set_description("Processing :"+dir1)
            path1 = os.path.join(self.PATH_AVDATA, dir1)

            for dir2 in tqdm(os.listdir(path1)):
                path2 = os.path.join(path1, dir2)

                for wav in glob.glob(path2+'/*.wav'):
                    fname = os.path.splitext(os.path.basename(wav))
                    label = re.findall('_.*' , fname[0])
                    
                    if len(label[0][1:]) < 3:
                        dir_save = os.path.join(self.PATH_SAVE, dir1, label[0][1:])
                        self.make_dir(dir_save)
                        shutil.copyfile(wav, os.path.join(dir_save, fname[0]+fname[1]))
        print('prepare_dataset was Done.')
                         
    #ディレクトリラベル付け
    def rename_dir_to_label(self, path_dir, path_dic):
        print('running rename_dir_to_label...')
        if os.path.exists(path_dic):
            f = open(path_dic,'rb')
            label_dic = pickle.load(f)
            
        else:
            label_dic = {}
            i = 0
            for dir1 in os.listdir(path_dir):
                path1 = os.path.join(self.PATH_SAVE, dir1)
                for dir2 in os.listdir(path1):
                    if not dir2 in label_dic:
                        label_dic[dir2] = i
                        i += 1

            f = open(path_dic,'wb')
            pickle.dump(label_dic,f)
            f.close
        
        for dir1 in os.listdir(path_dir):
            path1 = os.path.join(self.PATH_SAVE, dir1)
            for dir2 in os.listdir(path1):
                path2 = os.path.join(path1, dir2)
                rname = os.path.join(path1, str(label_dic[dir2]))
                os.rename(path2, rname)
                
        print('rename_to_dir_to_label was Done.')
                
    #MFCC抽出
    def extract_mfcc(self, path_dir):
        print('running extract_mfcc...')
        pbar1 = tqdm(os.listdir(path_dir))
        for dir1 in pbar1:
            pbar1.set_description("Processing :"+dir1)
            path1 = os.path.join(self.PATH_SAVE, dir1)
            for dir2 in tqdm(os.listdir(path1)):
                path2 = os.path.join(path1, dir2)
                for wav in glob.glob(path2+'/*.wav'):
                    fname = os.path.splitext(os.path.basename(wav))
                    #self.run_hcopy(self.PATH_CONFIG, wav, os.path.join(path2, fname[0]+'.mfc'))
                    self.run_hlist(self.MFCC_DIM, os.path.join(path2, fname[0]+'.mfc'), os.path.join(path2, fname[0]+'.mfctxt'))
        print('extract_mfcc was Done.')
    
    #TFRecordsファイルの作成
    def make_TFR(self, path_dir, path_tfr):
        print('running make_TFR...')
        pbar1 = tqdm(os.listdir(path_dir))
        with tf.python_io.TFRecordWriter(path_tfr) as writer:
            for dir1 in pbar1:
                pbar1.set_description("Processing :"+dir1)
                path1 = os.path.join(path_dir, dir1)
                for mfctxt in glob.glob(path1+'/*.mfctxt'):
                    with open(mfctxt, encoding='utf-8') as f:
                        lines = f.readlines()[1:-1]
                        data = np.array([])
                        for line in lines:
                            txt = line.strip()
                            txt = txt.split()
                            for val in txt[1:]:
                                data = np.append(data, float(val))
                        data = self.zero_padding(data, self.LEN_SEQ, self.MFCC_DIM)
                        data = data.astype(np.float32)
                        data = np.reshape(data, [-1, self.MFCC_DIM])
                        record = tf.train.Example(features=tf.train.Features(feature={
                            "label": tf.train.Feature(
                                int64_list=tf.train.Int64List(value=[int(dir1)])),
                            "data": tf.train.Feature(
                                bytes_list=tf.train.BytesList(value=[data.tobytes()])),
                        }))
                        writer.write(record.SerializeToString())
                        
        print('make_TFR was Done.')

In [ ]:
pd = PrepareDataset()
#pd.prepare_dataset()
#pd.rename_dir_to_label(pd.PATH_SAVE, pd.PATH_LABEL_TABLE)
#pd.extract_mfcc(pd.PATH_SAVE)
pd.make_TFR(os.path.join(pd.PATH_SAVE, 'train-male'), os.path.join(pd.PATH_SAVE, 'train-male.tfrecords'))
pd.make_TFR(os.path.join(pd.PATH_SAVE, 'train-female'), os.path.join(pd.PATH_SAVE, 'train-female.tfrecords'))
pd.make_TFR(os.path.join(pd.PATH_SAVE, 'test-male'), os.path.join(pd.PATH_SAVE, 'test-male.tfrecords'))
pd.make_TFR(os.path.join(pd.PATH_SAVE, 'test-female'), os.path.join(pd.PATH_SAVE, 'test-female.tfrecords'))

Processing :0:   0%|          | 0/22 [00:00<?, ?it/s]

running make_TFR...


Processing :0:   0%|          | 0/22 [00:00<?, ?it/s]

make_TFR was Done.
running make_TFR...


Processing :0:   0%|          | 0/22 [00:00<?, ?it/s]

make_TFR was Done.
running make_TFR...


Processing :11:  14%|█▎        | 3/22 [00:01<00:10,  1.78it/s]